In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.normalization import LocalResponseNorm
devices = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(devices)

cuda:0


### **Dataset**

In [ ]:
# 데이터 transform 정의
transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomCrop(227),
                                transforms.ToTensor(),
                                # transforms.RandomHorizontalFlip(1),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                
])

In [ ]:
# 데이터셋 저장
ci_train = datasets.CIFAR10('~/data/', train=True, transform=transform,download=True)
ci_test = datasets.CIFAR10('~/data/', train=True, transform=transform,download=True)
print("ci_train:\n", ci_train,'\n')
print("ci_test:\n", ci_test,'\n')

### **Data Iterator**

In [ ]:
batch_size = 256
train_iter = torch.utils.data.DataLoader(ci_train, batch_size=batch_size, shuffle=True, num_workers=1)
test_iter = torch.utils.data.DataLoader(ci_train, batch_size=batch_size, shuffle=True, num_workers=1)

In [ ]:
#이미지 확인하기 
def imshow(img):
  img = img / 2 + 0.5 
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show() 

# 학습용 이미지 뽑기
dataiter = iter(train_iter)
images, labels = dataiter.next() 

# 이미지 보여주기 
imshow(torchvision.utils.make_grid(images)) 
torchvision.utils.make_grid(images).shape

# 이미지별 라벨 (클래스) 보여주기 
# print(' '.join('%5s' % [labels[j]] for j in range(4)))



### **Model**

In [ ]:
class AlexNet(nn.Module):
  def __init__(self, input_size=227, num_classes = 10):
    super(AlexNet, self).__init__()

  # Convolution layer 정의
    self.layers = nn.Sequential(
        # 1 conv
        nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,padding=0),
        nn.ReLU(inplace=True),
        # nn.LocalResponseNorm(size =5, alpha=0.0001, beta=0.75, k=2),
        nn.BatchNorm2d(96),
        nn.MaxPool2d(kernel_size=3,stride=2),

        # 2 conv
        nn.Conv2d(in_channels=96, out_channels=256,kernel_size=5,stride=1,padding=2),
        nn.ReLU(inplace=True),
        # nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.BatchNorm2d(256),
        nn.MaxPool2d(kernel_size=3,stride=2),

        # 3 conv
        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        

        # 4 conv
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),


        # 5 conv
        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    self.fclayer = nn.Sequential(
        # 6 fc layer
        # nn.Dropout(0.5),
        nn.Linear(6*6*256, 4096),
        nn.ReLU(inplace=True),
        
        
        # 7 fc layer
        # nn.Dropout(0.5),
        nn.Linear(4096,4096),
        nn.ReLU(inplace=True),
       

        # 8 fc layer
        nn.Linear(4096,num_classes)
    )

    # 표준화 및 bias 초기화
    for layer in self.layers:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean=0, std=0.01)
        nn.init.constant_(layer.bias,0)

    # nn.init.constant_(self.layers[4].bias,1)
    # nn.init.constant_(self.layers[10].bias,0.8)
    # nn.init.constant_(self.layers[12].bias,0.6)
      
  def forward(self, train):
    output = self.layers(train)
    output = torch.flatten(output,1)
    output = self.fclayer(output)
      
    return output


In [ ]:
alexnet = AlexNet(227,10)
alexnet.to(devices)

AlexNet(
  (layers): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fclayer): Sequent

In [ ]:
optimizer = optim.SGD(alexnet.parameters(),lr=0.01,momentum=0.9, weight_decay = 0.0005)
criterion = nn.CrossEntropyLoss().to(devices)

In [ ]:
from tqdm.notebook import tqdm
import time

start_time = time.time()
min_loss = int(1e9)
history = []
accuracy = []

for epoch in range(20):
    epoch_loss = 0
    tk0 = tqdm(train_iter, total=len(train_iter),leave=False)
    for step, (inputs, labels) in enumerate(tk0, 0):
        inputs, labels = inputs.to(devices), labels.to(devices)
        outputs = alexnet(inputs)
        optimizer.zero_grad()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        history.append(loss.item())
    
    class_correct = list(0. for i in range(1000))
    class_total = list(0. for i in range(1000))
    
    with torch.no_grad():
        for data in test_iter:
            images, labels = data
            images = images.to(devices)
            labels = labels.to(devices)
            outputs = alexnet(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(labels.size()[0]):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1

    accuracy.append(sum(class_correct) /sum(class_total)*100)
    tqdm.write("[Epoch : %d] train_loss : %.5f test_acc: %.2f" % 
               (epoch +1 , epoch_loss/196, sum(class_correct) /sum(class_total)*100))
    
    if min_loss < epoch_loss:
      count+=1
      if count > 10:
        for g in optimizer.param_groups:
          g['lr']/=10
    else:
      min_loss = epoch_loss
      count = 0

print(time.time()-start_time)
print("Done")

In [ ]:
plt.plot(history)